<a href="https://colab.research.google.com/github/syedhammadahmed/quantumcomputings20/blob/master/quantum_computing_hw_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW 3

### Problem 1 

**Fourier matrix**

Let $\omega=\mathrm{exp}(2\pi i/N)$ be the $N$th root of unity.  The Fourier matrix of size $N$ is the matrix $F_N=(q_{k,\ell})\in\mathbb{C}^{N\times N}$ whose entries are given by
$$q_{k,\ell}=\frac{1}{\sqrt{N}} \omega^{k \cdot \ell}$$ for $k, \ell \in \{0,\ldots,N-1\}$.

Using numpy, implement the function ```get_fourier_matrix``` that creates the Fourier matrix of size $N$.

Compute the eigenvalues of eigenvectors of the Fourier matrix.  

To check that your code is correct, you should verify the following properties:

- the Fourier matrix is unitary 
- the fourth power of $F_N^4$ is equal to the identity matrix
- its eigenvalues are $1, i, -1, -i$ (the multiplicities of these eigenvalues are given on the wikipedia page [discrete fourier transform](https://en.wikipedia.org/wiki/Discrete_Fourier_transform) in section *eigenvalues and eigenvectors*)

**Cyclic shift matrix**

The cyclic shift matrix is the matrix $P_N=(p_{k,\ell}\in\mathbb{C}^{N\times N}$ whose entries are given by

$$ p_{k,\ell} = 1 \mbox{ if } \ell = k + 1 \pmod{N} $$

$$ p_{k,\ell} = 0 \mbox{ if } \ell \neq k + 1 \pmod{N} $$

Using numpy, implement the function ```get_cyclic_shift_matrix``` that create the cyclic shift matrix of size $N$.

Compute the eigenvalues of $P_N$.

To show that your code is correct, you should verify the following properties:

- the cyclic shift matrix is unitary
- the $N$th power of the cyclic shift matrix is the identity matrix
- the eigenvalues of the cyclic shift matrix are $N$th roots of unity
- the cyclic shift matrix is diagonalized by the Fourier matrix, that is, $F_N^\dagger P_N F_N$ is diagonal.

In [0]:
import math
import cmath 
import numpy as np
import numpy.linalg as NPLA

# author: Syed Hammad Ahmed (5068474)

# to print values to this significant figures
roundTo = 4

#verbose mode ON or OFF
verbose = 0


# returns the n-th root of unity power k*l (Fourier matrix element [k][l])
def getNthRootMatrixValue(k, l, n):
  real = math.cos(2 * (k * l) * math.pi * 1.0/n)
  im = math.sin(2 * (k * l) * math.pi * 1.0/n)
  return (1.0/math.sqrt(n))*complex(real, im)

# returns fourier matrix where n is the argument
# n is the order of the matrix
def get_fourier_matrix(n):
  fourierMatrix = np.zeros((n,n), dtype=complex)
  for k in range(0, n):
    for l in range(0, n): 
      fourierMatrix[k][l] = getNthRootMatrixValue(k, l, n)
  return fourierMatrix

# returns the eigen values of a matrix
def getEigenValNVec(matrix):
  return NPLA.eigvals(matrix)

# returns the dagger (complex conjugate) of a matrix
def getDagger(matrix):
  return np.conj(matrix)

# returns the inverse of a matrix
def getInverse(matrix):
  return NPLA.inv(matrix)
  
# returns matrix^k
def getMatrixPowerK(matrix, k):
  return NPLA.matrix_power(matrix, k)

# returns identity matrix of order k
def identityK(k):
  return np.eye(k, dtype=complex)

# returns True if a matrix is unitary, False otherwise
def verifyUnitary(matrix):
  isUnitary = False
  matrixDagger = getDagger(matrix)
  matrixInverse = getInverse(matrix)
  if verbose:
    print("Dagger (Complex Conjugate):")
    print(np.round(matrixDagger, roundTo))
    print("Inverse:")
    print(np.round(matrixInverse, roundTo))
  # rounded to set same precision for both matrices
  if np.array_equal(np.round(matrixDagger, roundTo), np.round(matrixInverse, roundTo)):
    isUnitary = True
  return isUnitary  

# verifies if the k-th power of a matrix is equal to the identity matrix
def verifyKthPowerIdentity(matrix, k):
  isKthPowerIdentity = False
  matrixPowerK = getMatrixPowerK(matrix, k) 
  n = np.size(matrix, 0)
  identity = identityK(n)
  if verbose:
    print("matrix Power {}: ".format(k))
    print(np.round(matrixPowerK, roundTo))
    print("Identity:")
    print(identity)
  # rounded to set same precision for both matrices  
  if np.array_equal(np.round(matrixPowerK, roundTo), np.round(identity, roundTo)):
    isKthPowerIdentity = True
  return isKthPowerIdentity  

# verifies if eigenvalues of a matrix are 1, i, −1, −i
def verifyEigenValuesUnit(eigenValues):
  isEigenValuesUnit = False
  n = np.size(eigenValues)
  ones = np.ones(n)
  eigenValuesPower4 = np.power(eigenValues, 4)
  if verbose:
    print("eigenValues:", eigenValues)
    # print("eigenValues^4:", eigenValuesPower4)
    # print("Each eigen values power 4 == 1. Hence, 4-th roots of unity!")
  if np.array_equal(eigenValuesPower4, ones):
    isEigenValuesUnit = True
  return isEigenValuesUnit

# verifies if eigenvalues of a matrix are n-th roots of unity
def verifyEigenValuesNthRootsOfUnity(eigenValues, n):
  isEigenValuesNthRootsOfUnity = False
  n = np.size(eigenValues)
  ones = np.ones(n, dtype=complex)
  eigenValuesPowerN = np.power(eigenValues, n)
  eigenValuesPowerN = np.round(eigenValuesPowerN, roundTo-1)
  if verbose:
    print("eigenValues:", eigenValues)
    print("eigenValues^n:", eigenValuesPowerN)
  if np.array_equal(eigenValuesPowerN, np.round(ones, roundTo)):
    isEigenValuesNthRootsOfUnity = True
    print("Each (eigen-value power n) == 1. Hence, n-th roots of unity!")

  return isEigenValuesNthRootsOfUnity

 
# returns cyclic shift matrix where n is the argument
# n is the order of the matrix
# p[k,ℓ] = 1 if ℓ=k+1(modN) 
# p[k,ℓ] = 0 if ℓ≠k+1(modN)
def get_cyclic_shift_matrix(n):
  cyclicShiftMatrix = np.zeros((n,n), dtype=complex)
  for k in range(0, n):
    for l in range(0, n): 
      cyclicShiftMatrix[k][l] = int(l == ((k + 1) % n))
  return cyclicShiftMatrix

def roundIt(matrix):
  return np.round(matrix, roundTo)

def isDiag(m):
  rows, cols = np.shape(m)
  for i in range(0, rows):
    for j in range(0, cols):
      if (i!=j) and (m[i][j] != complex(0, 0)):
        print ("{}, {}".format(i, j))
        return False
  return True
    
# the cyclic shift matrix is diagonalized by the Fourier matrix, that is,  F†NPNFN  is diagonal.
def verifyDiagonalizable(cyclicShiftMatrix, fourierMatrix):
  isDiagonalizable = False
  fourierMatrixDagger = getDagger(fourierMatrix)
  diagonalMatrix = np.matmul(np.matmul(roundIt(fourierMatrixDagger), roundIt(cyclicShiftMatrix)), roundIt(fourierMatrix))
  if verbose:  
    print(roundIt(diagonalMatrix))
  if isDiag(roundIt(diagonalMatrix)):
    isDiagonalizable = True
  return isDiagonalizable

#test driver code tests for values of n = 4 to 1
def main():
  i = 4
  while(i):
    print("n =",i)
    fourierMatrix = get_fourier_matrix(i)
    print("Fourier Matrix:")
    print(np.round(fourierMatrix, roundTo))

    isUnitary = verifyUnitary(fourierMatrix)
    print("Fourier matrix is unitary? i.e. Dagger == Inverse? : ", end =" ")
    if isUnitary:
      print("TRUE")
    else:
      print("FALSE")

    is4thPowerIdentity = verifyKthPowerIdentity(fourierMatrix, 4)
    print("fourierMatrix^4 == Identity? : ", end =" ")
    if is4thPowerIdentity:
      print("TRUE")
    else:
      print("FALSE")

    eigenValues = np.round(getEigenValNVec(fourierMatrix), roundTo)
    # isEigenValuesUnit = verifyEigenValuesUnit(eigenValues)
    isEigenValuesUnit = verifyEigenValuesUnit(eigenValues)
    print("Fourier Matrix eigen values lie on complex unit circle? : ", end =" ")
    if isEigenValuesUnit:
      print("TRUE")
    else:
      print("FALSE")

    # cyclic shift matrix
    print("Cyclic Shift Matrix:")
    cyclicShiftMatrix = get_cyclic_shift_matrix(i)
    print(cyclicShiftMatrix)
    
    eigenValues = np.round(getEigenValNVec(cyclicShiftMatrix), roundTo)
  # the cyclic shift matrix is unitary
    isUnitary = verifyUnitary(cyclicShiftMatrix)
    print("Cyclic Shift matrix is unitary? i.e. Dagger == Inverse? : ", end =" ")
    if isUnitary:
      print("TRUE")
    else:
      print("FALSE")

  # the N th power of the cyclic shift matrix is the identity matrix
    isNthPowerIdentity = verifyKthPowerIdentity(cyclicShiftMatrix, i)
    print("cyclicShiftMatrix^{} == Identity? : ".format(i), end =" ")
    if isNthPowerIdentity:
      print("TRUE")
    else:
      print("FALSE")

  # the eigenvalues of the cyclic shift matrix are N th roots of unity
    isEigenValuesUnit = verifyEigenValuesNthRootsOfUnity(eigenValues, i)
    print("Cyclic Shift matrix are Nth roots of unity? : ", end =" ")
    if isEigenValuesUnit:
      print("TRUE")
    else:
      print("FALSE")

  # the cyclic shift matrix is diagonalized by the Fourier matrix, that is,  F†NPNFN  is diagonal.
    isDiagonalizable = verifyDiagonalizable(cyclicShiftMatrix, fourierMatrix)
    print("Cyclic Shift matrix diagonalized by the Fourier matrix? : ", end =" ")
    if isDiagonalizable:
      print("TRUE")
    else:
      print("FALSE")


    i = i - 1
    print("\n"*2)


if __name__ == "__main__":
    main()

n = 4
Fourier Matrix:
[[ 0.5+0.j   0.5+0.j   0.5+0.j   0.5+0.j ]
 [ 0.5+0.j   0. +0.5j -0.5+0.j  -0. -0.5j]
 [ 0.5+0.j  -0.5+0.j   0.5-0.j  -0.5+0.j ]
 [ 0.5+0.j  -0. -0.5j -0.5+0.j   0. +0.5j]]
Fourier matrix is unitary? i.e. Dagger == Inverse? :  TRUE
fourierMatrix^4 == Identity? :  TRUE
Fourier Matrix eigen values lie on complex unit circle? :  TRUE
Cyclic Shift Matrix:
[[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Cyclic Shift matrix is unitary? i.e. Dagger == Inverse? :  FALSE
cyclicShiftMatrix^4 == Identity? :  TRUE
Each (eigen-value power n) == 1. Hence, n-th roots of unity!
Cyclic Shift matrix are Nth roots of unity? :  TRUE
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]
Cyclic Shift matrix diagonalized by the Fourier matrix? :  TRUE



n = 3
Fourier Matrix:
[[ 0.5774+0.j   0.5774+0.j   0.5774+0.j ]
 [ 0.577

### Problem 2

Using Qiskit IBM Q Experience, implement the quantum phase estimation circuit with 3-bit precision.  

This quantum circuit is explained on pages 110-113 (Inverse Fourier transform for 3 qubits) of the lecture slides.

Write a function that create the state

$$ 
\frac{1}{\sqrt{2}} ( |0\rangle + e^{2\pi i 2^2 \varphi} |1\rangle ) \otimes 
\frac{1}{\sqrt{2}} ( |0\rangle + e^{2\pi i 2^1 \varphi} |1\rangle ) \otimes 
\frac{1}{\sqrt{2}} ( |0\rangle + e^{2\pi i 2^0 \varphi} |1\rangle ) 
$$

for arbitrary $\varphi\in [0,1)$. 

(This is the state that we would obtain by 
- preparing the three control bits in the $|+\rangle$ state 
- applying the control power of $U^{2^2}, U^{2^1}, U^{2^0}$ gates

assuming that the target qubit is in state $|\psi\rangle$ and $U|\psi\rangle=e^{2 \pi i \varphi} |\psi\rangle$.)

You can preprare the initial state applying ```U1``` gates on qubits prepared $|+\rangle$. See [Qiskit U1 gate documentation](https://qiskit.org/documentation/stubs/qiskit.extensions.U1Gate.html). 

The controlled $R_k^\dagger$ gates occuring in the inverse Fourier transform circuit can be realized with the help of controlled ```U1``` gates.  

In [0]:
!pip install qiskit==0.16.2
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
# provider = IBMQ.load_account()

# Enable your account on Qiskit, replace 'My_API_Token' with your newly generated token
qiskit.IBMQ.save_account('f6500d0c95014269ca779482456925bf5a3236c4da2e8b9717e7e72af1a9a80a1e6736a0b27512b5416b0c8c74e7372b855b6e6359742b7528a237b57b3f9458')
qiskit.IBMQ.load_account()
# # After loading credentials we query the backends
# qiskit.IBMQ.backends()


# Useful additional packages
import matplotlib.pyplot as plt
import math
import cmath
import numpy as np

# author: Syed Hammad Ahmed (5068474)

def getTheta(bitNo, phi):
    return 2*pi*2**(3-bitNo-1)*phi

def initStates(phi):
    thetas = np.zeros(3)
    for i in range(3):
        thetas[i] = getTheta(i, phi)
    return thetas  
        
# def main():
backend = BasicAer.get_backend('unitary_simulator')
n = 3
q = QuantumRegister(n)
c = ClassicalRegister(n)
phi = 0.875

qc = QuantumCircuit(q, c)

# initilize states
thetas = initStates(phi)
for i in range(3):
    qc.h(q[i])
    qc.u1(getTheta(i, phi), q[i])

# inverse QFT     
qc.h(q[0])
qc.cu1(-pi/2, q[0], q[1])
qc.h(q[1])
qc.cu1(-pi/4, q[0], q[2])
qc.cu1(-pi/2, q[1], q[2])
qc.h(q[2])
qc.barrier()
qc.measure(q, c)
qc.draw()

# if __name__ == "__main__":
#     main()

     |████████████████████████████████| 2.9MB 2.8MB/s 
     |████████████████████████████████| 23.0MB 1.7MB/s 
     |████████████████████████████████| 143kB 49.7MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 1.8MB 46.3MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 5.6MB 48.1MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 296kB 46.4MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 24.3MB 126kB/s 
     |████████████████████████████████| 573kB 50.2MB/s 
     |████████████████████████████████| 2.7MB 41.3MB/s 
  Created wheel for qiskit: filename=qiskit-0.16.2-cp36-none-any.whl size=2898 sha256=e786aa8babadce3d9372080e14bb4e43d033f22444df55f0ee65c8cc9a571e7e
  Stored in directory: /root/.cache/pip/wheels/8d/eb/5a/ce0d4fa7714b1b698

NameError: ignored

In [0]:
import qiskit
qiskit.__version__

ModuleNotFoundError: ignored

In [0]:
import time
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit,  available_backends, execute, register, get_backend
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
Useful additional packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

# author: Syed Hammad Ahmed (5068474)

APItoken = 'f6500d0c95014269ca779482456925bf5a3236c4da2e8b9717e7e72af1a9a80a1e6736a0b27512b5416b0c8c74e7372b855b6e6359742b7528a237b57b3f9458'

qx_config = {
    "APItoken": APItoken,
    "url":"https://quantumexperience.ng.bluemix.net/api"}


#test driver code
def main():
  backend = BasicAer.get_backend('unitary_simulator')
  n = 3
  q = QuantumRegister(n)
  c = ClassicalRegister(n)

  qc = QuantumCircuit(q, c)

  qc.h(q[0])
  qc.cu1(-pi/2, q[0], q[1])
  qc.h(q[1])
  qc.cu1(-pi/2, q[0], q[2])
  qc.cu1(-pi/2, q[1], q[2])
  qc.h(q[2])
  qc.measure(q, c)
  qc.draw()


if __name__ == "__main__":
    main()


ImportError: ignored

In [0]:
import time
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit,  available_backends, execute, register, get_backend
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit import QuantumProgram

# Useful additional packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

# author: Syed Hammad Ahmed (5068474)

backend = 'ibmqx5'

APItoken = 'f6500d0c95014269ca779482456925bf5a3236c4da2e8b9717e7e72af1a9a80a1e6736a0b27512b5416b0c8c74e7372b855b6e6359742b7528a237b57b3f9458'

qx_config = {
    "APItoken": APItoken,
    "url":"https://quantumexperience.ng.bluemix.net/api"}


#test driver code
def main():
  qp = QuantumProgram()

  backend = BasicAer.get_backend('unitary_simulator')
  n = 3
  q = QuantumRegister(n)
  c = ClassicalRegister(n)

  qc = QuantumCircuit(q, c)
  qc = qp.create_circuit('HelloWorldCircuit', [q],[c])

  qc.h(q[0])
  qc.cu3(0, 0, -pi/2, q[0], q[1])
  qc.h(q[1])
  qc.cu3(0, 0, -pi/2, q[0], q[2])
  qc.cu3(0, 0, -pi/2, q[1], q[2])
  qc.h(q[2])
  qc.measure(q, c)
  qc.draw()

  qp.set_api(token,url='https://quantumexperience.ng.bluemix.net/api')

if __name__ == "__main__":
    main()


ModuleNotFoundError: ignored